## Dependencias

In [32]:
import pandas as pd
import json #JSON = Java Script Object Notation
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,DATE,CHAR,DATETIME, TIME
import os

pd.set_option('display.max_columns',None)

## Credenciales

In [33]:
creds = json.load(open('credenciales1.json','rb'))

## Crear conexión a base de datos

In [34]:
url = f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['servidor']}/retail"
cnx = create_engine(url,encoding='utf8')
cnx = cnx.connect()

In [35]:
cnx.closed

False

## Lectura y limpieza de datos

In [36]:
datos = pd.read_excel('Online Retail.xlsx')

In [37]:
datos.shape

(541909, 8)

In [38]:
datos.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Separación de Entidades

### Entidad Customer

In [95]:
entCustomer = datos[['CustomerID']].drop_duplicates().dropna().reset_index(drop=True)
entCustomer['CustomerID'] = entCustomer['CustomerID'].astype(int)
entCustomer

,CustomerID
0,17850
1,13047
2,12583
3,13748
4,15100
...,...
4367,13436
4368,15520
4369,13298
4370,14569


In [96]:
entCustomer.to_sql(con=cnx,
                   name='Customer',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'CustomerID':INTEGER})

4372

### Entidad Product

In [97]:
entProducto = datos[['StockCode','Description']].drop_duplicates().dropna().reset_index(drop=True).astype(str)
entProducto['StockCode'] = entProducto['StockCode'].map(str.upper)
entProducto = entProducto.groupby('StockCode').first().reset_index()
entProducto.to_sql(con=cnx,
                   name='Product',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'StockCode':VARCHAR(12),'Description':VARCHAR(35)})

3848

In [98]:
entProducto

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3843,GIFT_0001_50,Dotcomgiftshop Gift Voucher £50.00
3844,M,Manual
3845,PADS,PADS TO MATCH ALL CUSHIONS
3846,POST,POSTAGE


In [99]:
len(entProducto['StockCode'].unique())

3848

In [100]:
entProducto.loc[entProducto['StockCode']=='84509A']

,StockCode,Description
2957,84509A,SET OF 4 ENGLISH ROSE PLACEMATS


### Entidad Invoice

In [101]:
entInvoice = datos[['InvoiceNo','InvoiceDate','CustomerID']].drop_duplicates().dropna().copy().reset_index(drop=True)
entInvoice = entInvoice.sort_values(by=['InvoiceNo','InvoiceDate']).groupby('InvoiceNo').first().reset_index()
entInvoice

,InvoiceNo,InvoiceDate,CustomerID
0,536365,2010-12-01 08:26:00,17850.0
1,536366,2010-12-01 08:28:00,17850.0
2,536367,2010-12-01 08:34:00,13047.0
3,536368,2010-12-01 08:34:00,13047.0
4,536369,2010-12-01 08:35:00,13047.0
...,...,...,...
22185,C581484,2011-12-09 09:27:00,16446.0
22186,C581490,2011-12-09 09:57:00,14397.0
22187,C581499,2011-12-09 10:28:00,15498.0
22188,C581568,2011-12-09 11:57:00,15311.0


In [102]:
entInvoice['InvoiceDate'] = entInvoice['InvoiceDate'].astype(str)
entInvoice[['InvoiceDate_fecha', 'InvoiceDate_hora']] = entInvoice['InvoiceDate'].str.split(expand=True)
entInvoice

,InvoiceNo,InvoiceDate,CustomerID,InvoiceDate_fecha,InvoiceDate_hora
0,536365,2010-12-01 08:26:00,17850.0,2010-12-01,08:26:00
1,536366,2010-12-01 08:28:00,17850.0,2010-12-01,08:28:00
2,536367,2010-12-01 08:34:00,13047.0,2010-12-01,08:34:00
3,536368,2010-12-01 08:34:00,13047.0,2010-12-01,08:34:00
4,536369,2010-12-01 08:35:00,13047.0,2010-12-01,08:35:00
...,...,...,...,...,...
22185,C581484,2011-12-09 09:27:00,16446.0,2011-12-09,09:27:00
22186,C581490,2011-12-09 09:57:00,14397.0,2011-12-09,09:57:00
22187,C581499,2011-12-09 10:28:00,15498.0,2011-12-09,10:28:00
22188,C581568,2011-12-09 11:57:00,15311.0,2011-12-09,11:57:00


In [103]:
entFH=entInvoice[['InvoiceDate_fecha' ,'InvoiceDate_hora', 'InvoiceDate']].copy().drop_duplicates().reset_index(drop=True)
entFH['FH_id']=entFH.index+1
entFH

,InvoiceDate_fecha,InvoiceDate_hora,InvoiceDate,FH_id
0,2010-12-01,08:26:00,2010-12-01 08:26:00,1
1,2010-12-01,08:28:00,2010-12-01 08:28:00,2
2,2010-12-01,08:34:00,2010-12-01 08:34:00,3
3,2010-12-01,08:35:00,2010-12-01 08:35:00,4
4,2010-12-01,08:45:00,2010-12-01 08:45:00,5
...,...,...,...,...
20429,2011-12-09,09:27:00,2011-12-09 09:27:00,20430
20430,2011-12-09,09:57:00,2011-12-09 09:57:00,20431
20431,2011-12-09,10:28:00,2011-12-09 10:28:00,20432
20432,2011-12-09,11:57:00,2011-12-09 11:57:00,20433


In [104]:
entInvoice = pd.merge(entInvoice, entFH, how='left', on='InvoiceDate')
entInvoice = entInvoice.drop(['InvoiceDate', 'InvoiceDate_fecha_x','InvoiceDate_hora_x','InvoiceDate_fecha_y','InvoiceDate_hora_y'], axis=1)
entInvoice 

,InvoiceNo,CustomerID,FH_id
0,536365,17850.0,1
1,536366,17850.0,2
2,536367,13047.0,3
3,536368,13047.0,3
4,536369,13047.0,4
...,...,...,...
22185,C581484,16446.0,20430
22186,C581490,14397.0,20431
22187,C581499,15498.0,20432
22188,C581568,15311.0,20433


In [105]:
entFH = entFH.drop(['InvoiceDate'], axis=1)
entFH

,InvoiceDate_fecha,InvoiceDate_hora,FH_id
0,2010-12-01,08:26:00,1
1,2010-12-01,08:28:00,2
2,2010-12-01,08:34:00,3
3,2010-12-01,08:35:00,4
4,2010-12-01,08:45:00,5
...,...,...,...
20429,2011-12-09,09:27:00,20430
20430,2011-12-09,09:57:00,20431
20431,2011-12-09,10:28:00,20432
20432,2011-12-09,11:57:00,20433


In [106]:
entInvoice.to_sql(con=cnx,
                   name='Invoice',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'InvoiceNo':CHAR(7),
                           'FH_id':INTEGER,
                           'CustomerID':INTEGER
                         }
             )

22190

In [107]:
entInvoice

,InvoiceNo,CustomerID,FH_id
0,536365,17850.0,1
1,536366,17850.0,2
2,536367,13047.0,3
3,536368,13047.0,3
4,536369,13047.0,4
...,...,...,...
22185,C581484,16446.0,20430
22186,C581490,14397.0,20431
22187,C581499,15498.0,20432
22188,C581568,15311.0,20433


In [108]:
len(entInvoice['FH_id'].unique())

20434

### Entidad fecha

In [109]:
entFH.head(2)

,InvoiceDate_fecha,InvoiceDate_hora,FH_id
0,2010-12-01,08:26:00,1
1,2010-12-01,08:28:00,2


In [110]:
entFH.to_sql(con=cnx,
             name='Fecha-hora',
             if_exists='replace',
             index=False,
             chunksize=10000,
             dtype={'InvoiceDate_fecha':DATE,
                    'InvoiceDate_hora':TIME,
                    'FH_id':INTEGER})

20434

### Entidad Transaction

In [139]:
entTxn = datos[['InvoiceNo','StockCode','Quantity','UnitPrice','Country']].copy().dropna().reset_index(drop=True)
entTxn.insert(0,'TxnID', entTxn.index+1) 
entTxn

,TxnID,InvoiceNo,StockCode,Quantity,UnitPrice,Country
0,1,536365,85123A,6,2.55,United Kingdom
1,2,536365,71053,6,3.39,United Kingdom
2,3,536365,84406B,8,2.75,United Kingdom
3,4,536365,84029G,6,3.39,United Kingdom
4,5,536365,84029E,6,3.39,United Kingdom
...,...,...,...,...,...,...
541904,541905,581587,22613,12,0.85,France
541905,541906,581587,22899,6,2.10,France
541906,541907,581587,23254,4,4.15,France
541907,541908,581587,23255,4,4.15,France


In [140]:
entTxn['StockCode'] = entTxn['StockCode'].map(str).map(str.upper)
entTxn = entTxn.merge(entInvoice,on='InvoiceNo',how='inner')
print(entTxn.shape)
entTxn = entTxn.merge(entProducto,on='StockCode',how='inner')
print(entTxn.shape)
entTxn = entTxn.drop(['CustomerID','Description', 'FH_id'], axis=1)

(406829, 8)
(406829, 9)


In [141]:
entTxn

,TxnID,InvoiceNo,StockCode,Quantity,UnitPrice,Country
0,1,536365,85123A,6,2.55,United Kingdom
1,50,536373,85123A,6,2.55,United Kingdom
2,67,536375,85123A,6,2.55,United Kingdom
3,221,536390,85123A,64,2.55,United Kingdom
4,263,536394,85123A,32,2.55,United Kingdom
...,...,...,...,...,...,...
406824,530383,580865,90089,12,0.19,United Kingdom
406825,530393,580865,90089,12,0.19,United Kingdom
406826,538996,581425,90089,1,0.19,United Kingdom
406827,540422,581483,23843,80995,2.08,United Kingdom


In [142]:
len(entTxn['Country'].unique())

37

In [143]:
entCountry = entTxn[['Country']].copy().drop_duplicates().reset_index(drop=True)
entCountry['CountID']=entCountry.index + 1
entCountry.head(2)

,Country,CountID
0,United Kingdom,1
1,Spain,2


In [144]:
entTxn = pd.merge(entTxn, entCountry, how='left', on='Country')
entTxn = entTxn.drop(['Country'], axis=1)
entTxn

,TxnID,InvoiceNo,StockCode,Quantity,UnitPrice,CountID
0,1,536365,85123A,6,2.55,1
1,50,536373,85123A,6,2.55,1
2,67,536375,85123A,6,2.55,1
3,221,536390,85123A,64,2.55,1
4,263,536394,85123A,32,2.55,1
...,...,...,...,...,...,...
406824,530383,580865,90089,12,0.19,1
406825,530393,580865,90089,12,0.19,1
406826,538996,581425,90089,1,0.19,1
406827,540422,581483,23843,80995,2.08,1


In [145]:
entTxn.dropna().to_sql(con=cnx,
                   name='Transaction',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'TxnID':INTEGER,
                          'InvoiceNo':CHAR(7),
                          'StockCode':VARCHAR(12),
                          'Quantity':INTEGER,
                          'CountID':INTEGER,
                          'UnitPrice':FLOAT})

406829

In [146]:
max(entCountry['Country'].apply(len))

20

In [147]:
len(entTxn['CountID'].unique())

37

### Entidad pais

In [148]:
entCountry.head(2)

,Country,CountID
0,United Kingdom,1
1,Spain,2


In [149]:
entCountry.dropna().to_sql(con=cnx,
                   name='Country',
                   if_exists='replace',
                   index=False,
                   chunksize=10000,
                   dtype={'Country':VARCHAR(20),
                          'CountID':INTEGER})

37